<a href="https://colab.research.google.com/github/andresvosa/testNotebooks/blob/main/words_and_bytes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def int_to_bytes(x: int) -> bytes:
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')
    
def int_from_bytes(xbytes: bytes) -> int:
    return int.from_bytes(xbytes, 'big')

In [2]:
def ieee_754_conversion(n, sgn_len=1, exp_len=8, mant_len=23):
    """
    Converts an arbitrary precision Floating Point number.
    Note: Since the calculations made by python inherently use floats, the accuracy is poor at high precision.
    :param n: An unsigned integer of length `sgn_len` + `exp_len` + `mant_len` to be decoded as a float
    :param sgn_len: number of sign bits
    :param exp_len: number of exponent bits
    :param mant_len: number of mantissa bits
    :return: IEEE 754 Floating Point representation of the number `n`
    """
    if n >= 2 ** (sgn_len + exp_len + mant_len):
        raise ValueError("Number n is longer than prescribed parameters allows")

    sign = (n & (2 ** sgn_len - 1) * (2 ** (exp_len + mant_len))) >> (exp_len + mant_len)
    exponent_raw = (n & ((2 ** exp_len - 1) * (2 ** mant_len))) >> mant_len
    mantissa = n & (2 ** mant_len - 1)

    sign_mult = 1
    if sign == 1:
        sign_mult = -1

    if exponent_raw == 2 ** exp_len - 1:  # Could be Inf or NaN
        if mantissa == 2 ** mant_len - 1:
            return float('nan')  # NaN

        return sign_mult * float('inf')  # Inf

    exponent = exponent_raw - (2 ** (exp_len - 1) - 1)

    if exponent_raw == 0:
        mant_mult = 0  # Gradual Underflow
    else:
        mant_mult = 1

    for b in range(mant_len - 1, -1, -1):
        if mantissa & (2 ** b):
            mant_mult += 1 / (2 ** (mant_len - b))

    return sign_mult * (2 ** exponent) * mant_mult


In [3]:
integer_input = 3948036974
short_int = 65535
to_bytes = int_to_bytes(integer_input)
#print(to_bytes)
print(to_bytes.hex())
print(int(int_to_bytes(integer_input).hex(), base=16))
print(integer_input.to_bytes(4, 'little'))


eb52436e
3948036974
b'nCR\xeb'


In [4]:
#print(type(to_bytes))
#help(int.to_bytes)
#help(bytes)
#help(int.from_bytes)
#help(int_to_bytes)
#dir(bytearray)
help(bytearray)

Help on class bytearray in module builtins:

class bytearray(object)
 |  bytearray(iterable_of_ints) -> bytearray
 |  bytearray(string, encoding[, errors]) -> bytearray
 |  bytearray(bytes_or_buffer) -> mutable copy of bytes_or_buffer
 |  bytearray(int) -> bytes array of size given by the parameter initialized with null bytes
 |  bytearray() -> empty bytes array
 |  
 |  Construct a mutable bytearray object from:
 |    - an iterable yielding integers in range(256)
 |    - a text string encoded using the specified encoding
 |    - a bytes or a buffer object
 |    - any object implementing the buffer API.
 |    - an integer
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __alloc__(...)
 |      B.__alloc__() -> int
 |      
 |      Return the number of bytes actually allocated.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |   

In [5]:
# Raw bytes manipulations using bytearray class object
recieved_value = 3948036974 # Word 4 bytes length (32 bits)
type(recieved_value.to_bytes(4, 'big'))
byte_arr = bytearray(recieved_value.to_bytes(4, 'big'))
#type(byte_arr)
print(recieved_value, byte_arr.hex())
print([bait for bait in byte_arr]) # Print out bytes decimal values
#byte_arr[0], byte_arr[2] = byte_arr[2], byte_arr[0]
#byte_arr[1], byte_arr[3] = byte_arr[3], byte_arr[1]
# Change LSW <-> MSW can be done in one row
byte_arr[0], byte_arr[2], byte_arr[1], byte_arr[3] = byte_arr[2], byte_arr[0], byte_arr[3], byte_arr[1] 
print([bait for bait in byte_arr]) # Print out bytes decimal values
reordered_int = int(byte_arr.hex(), base=16)
print(reordered_int, byte_arr.hex())
res_float = ieee_754_conversion(reordered_int)
print(res_float)

3948036974 eb52436e
[235, 82, 67, 110]
[67, 110, 235, 82]
1131342674 436eeb52
238.91921997070312
